In [1]:
import sklearn.ensemble
import shap
import numpy as np
import numba
import time
import xgboost

/home/jarek/projects/shap/venv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
X,y = shap.datasets.boston()
X.shape



Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np

        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California housing dataset (i.e.
    :func:`~sklearn.datasets.fetch_california_housing`) and the Ames housing
    datase

(506, 13)

In [3]:
model = sklearn.ensemble.RandomForestRegressor(n_estimators=1000, max_depth=4)
model.fit(X, y)



RandomForestRegressor(max_depth=4, n_estimators=1000)

In [4]:
bst = xgboost.train({"learning_rate": 0.01, "max_depth": 4}, xgboost.DMatrix(X, label=y), 1000)

In [5]:
start = time.time()
shap_values = bst.predict(xgboost.DMatrix(X), pred_contribs=True)
print(time.time() - start)


0.08113360404968262


In [ ]:
from shap.explainers.pytree import *
# x = np.ones(X.shape[1])
# TreeExplainer(model).shap_values(x)

In [ ]:
start = time.time()
ex = TreeExplainer(model)
print(time.time() - start)
start = time.time()
ex.shap_values(X.iloc[:,:])
print(time.time() - start)

In [8]:
from shap import TreeExplainer as TreeCext
start = time.time()
t = TreeCext(model).shap_values(X)
print(time.time() - start)

ile drzew cext? 1000
using treeshap and cext
banz = False
here37
using dense_Tree_pred
0.4581146240234375


In [9]:
start = time.time()
t = TreeCext(model).shap_values(X, banz=True)
print(time.time() - start)

ile drzew cext? 1000
using treeshap and cext
banz = True
0.019055843353271484
using cext banz
